## TẠO CHATBOT ĐƠN GIẢN BẰNG NATURAL LANGUAGE TOOLKIT (NLTK)
*In courtesy of Parul Pandey (parulnith)*

- *Tạo chatbot dựa trên mẫu truy hồi (retrieval-based model)*
- *Mô hình này lấy câu trả lời/ hồi đáp từ một tập định trước bằng một số kỹ thuật khớp luật (heuristics) cách diễn đạt ngôn từ (expression matching) để có thế nhận được phản hồi phù hợp nhất với đầu vào (input) và bối cảnh (context).*

### **Tải NLTK và NLTK downloader trên terminal**

In [ ]:
pip install nltk

In [ ]:
#Test installation bằng cách 
#mở python 
python3
#đưa nltk vào script
import nltk
from nltk.stem import WordNetLemmatizer

Tải các packages cần thiết 

In [ ]:
nltk.download('popular', quiet=True) # tải popular packages
# nltk.download('punkt') 
# nltk.download('wordnet') 

Hoặc tải hết

In [ ]:
nltk.download()

### **Import các thư viện cần thiết**

In [ ]:
import numpy as np
import io # read input write output
import random # generate random numbers
import string # to process standard python strings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### **Đọc văn bản**

- Ở bài này, chúng ta dùng một đoạn từ trang wikipedia về chatbots để làm dữ liệu văn bản (corpus)
- Lưu đoạn dữ liệu vào file .txt và đặt tên 'chatbot.txt'
- Tuy nhiên, có thể dùng bất cứ văn bản nào và lưu tên theo ý mình 

In [ ]:
f=open('chatbot.txt','r',errors = 'ignore')
raw=f.read()
raw=raw.lower()# converts to lowercase

Chuyển hết các từ thành chữ viết thường để thuật toán coi các từ giống nhau nhưng khác cách viết viết hoa/viết thường là một

### **Tokenisation**
Cho các từ và câu vào thành dãy các tokens câu, tokens từ

In [ ]:
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

*test:*

In [ ]:
sent_tokens[:2]
['a chatbot (also known as a talkbot, chatterbot, bot, im bot, interactive agent, or artificial conversational entity) is a computer program or an artificial intelligence which conducts a conversation via auditory or textual methods.',
 'such programs are often designed to convincingly simulate how a human would behave as a conversational partner, thereby passing the turing test.']
word_tokens[:2]
['a', 'chatbot', '(', 'also', 'known']

### **Tiền xử lí văn bản**

Định nghĩa 1 hàm tên là LemTokens, lấy vào các tokens, (cho các từ giống nhau vào thành một nhóm), cho ra các từ chung của nhóm đó

In [ ]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

### **Nối các từ khoá**

Định nghĩa 1 hàm để chào hỏi bot. Khi chúng nhập câu chào, bot sẽ chào lại chúng ta  :v

In [ ]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

### **Tạo câu trả lời**

More explanation about:

- Bag of words

- TF-IDF Approach (term frequency - inverse document frequency)

- Cosine Similarity

In [ ]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

More explanation down here

In [ ]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")